# CorrelAid for Klima*Kollektiv Karte der Transportleitung, Rohdungstrassen und Zerrstörungspunkte entlang der Trasse
Sep 27, 2024

by Tim Appelhans

In [767]:
# !pip install geopandas
# !pip install folium
# !pip install requests

In [768]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon, MultiPolygon, MultiLineString
import json

In [769]:
# ---- Visualization ----

# Create a base map
m = folium.Map(location = [51.0, 6.5], zoom_start = 10,  min_zoom = 8)

# Set up custome map panes for points, lines, polygons
folium.map.CustomPane("pointsPane", z_index = 330).add_to(m)
folium.map.CustomPane("linesPane", z_index = 320).add_to(m)
folium.map.CustomPane("polygonsPane", z_index = 310).add_to(m)


Add `Esri.WorldImagery` tile layer (see [Leaflet Provider Demo](https://leaflet-extras.github.io/leaflet-providers/preview/) for a quick preview of various basemaps).

In [770]:
# add Esri World Imagery tile layer
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri.WorldImagery',
    overlay=False,
    control=True,
    min_zoom = 8,
    show=False # hide layer when opening the map
).add_to(m)

# add grey canvas tile layer
folium.TileLayer(
    tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png',
    attr='Carto',
    name='Grey Canvas',
    overlay=False,
    control=True,
    min_zoom = 8,
    show=True # hide layer when opening the map
).add_to(m)



In [771]:
hambacher_see = gpd.read_file("/home/tim/Downloads/hambachersee_gp.gpkg", layer = 'hambacher_see_wasser')
hambacher_see = hambacher_see.set_geometry([Polygon(hambacher_see.get_coordinates())])
hambacher_see
#hambacher_see.geometry = hambacher_see.geometry.apply(lambda x: Polygon(x.get_coordinates()))


,id,geometry
0,0,"POLYGON ((6.49395 50.88401, 6.48964 50.88573, ..."


In [772]:
garzweiler_see = gpd.read_file("/home/tim/Downloads/garzweilersee_wasser.gpkg", layer = 'garzweilersee_wasser')
garzweiler_see = garzweiler_see.set_geometry([Polygon(garzweiler_see.get_coordinates())])
inder_see = gpd.read_file("/home/tim/Downloads/indersee_gp.gpkg", layer = 'inder_see_wasser')
inder_see = inder_see.set_geometry([Polygon(inder_see.get_coordinates())])

In [773]:
folium.GeoJson(
    hambacher_see, 
    name="Hambacher See", 
    pane = "polygonsPane",
    style_function = lambda feature: {
        'color': '#4e4eff',
        'fillColor': "#4e4eff",
        'fillOpacity': 0.8,
        'fill': True,
        'weight': 0
    },
    tooltip = "Hambacher See"
).add_to(m)

folium.GeoJson(
    garzweiler_see, 
    name="Garzweiler See", 
    pane = "polygonsPane",
    style_function = lambda feature: {
        'color': '#4e4eff',
        'fillColor': "#4e4eff",
        'fillOpacity': 0.8,
        'fill': True,
        'weight': 0
    },
    tooltip = "Garzweiler See"
).add_to(m)

folium.GeoJson(
    inder_see, 
    name="Inder See", 
    pane = "polygonsPane",
    style_function = lambda feature: {
        'color': '#4e4eff',
        'fillColor': "#4e4eff",
        'fillOpacity': 0.8,
        'fill': True,
        'weight': 0
    },
    tooltip = "Inder See"
).add_to(m)

In [774]:
# load GeoPackage file 'transportleitungen_geom' into geodataframe gdf
pipelines = gpd.read_file("transportleitung_geom.gpkg")

# print to inspect the data
pipelines.Name = ["Transportleitung West", "Transportleitung Süd"]
pipelines

,id,Name,geometry
0,0,Transportleitung West,"MULTILINESTRING ((350531.177 5662445.558, 3501..."
1,1,Transportleitung Süd,"MULTILINESTRING ((333215.541 5658810.732, 3341..."


In [775]:
pipelines = pipelines.to_crs("EPSG:4326") # change CRS to the same as in 'consumer'
pipelines.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [776]:

# Add the GeoDataFrame to the map
folium.GeoJson(
    pipelines, 
    name="Water pipelines", # again, determine the name for the layer toggle
    pane = "linesPane",
    style_function = lambda feature: {
        'color': '#707070' if feature['properties']['Name'] == 'transportleitung_sued' else '#3f3f3f',
        'weight': 5
    },
    tooltip = folium.GeoJsonTooltip(fields=['Name'], labels = False)
).add_to(m)


In [777]:
# load GeoPackage file for clearing
clearing = gpd.read_file("/home/tim/Downloads/rodungstrasse.gpkg")

# print to inspect the data
clearing

,pkuid,geometry
0,1,"POLYGON ((6.86325 51.09179, 6.86972 51.09416, ..."


In [778]:
# Add the GeoDataFrame to the map
folium.GeoJson(
    data = clearing, 
    name = "Clearing", # again, determine the name for the layer toggle
    pane = "polygonsPane",
    style_function = lambda feature: {
        'color': 'brown',
        'weight': 1,
        'fillColor': 'brown',
        'fillOpacity': 0.3         
    },tooltip = "Rodungstrasse"
).add_to(m)

In [779]:
# load GeoPackage file 'transportleitungen_geom' into geodataframe gdf
destructions = gpd.read_file("/home/tim/Downloads/Rheinwassertransportleitung.kml")

polys = destructions[destructions.geom_type=='Polygon']
points = destructions[destructions.geom_type=='Point']

# print to inspect the data
polys
points

,Name,Description,geometry
0,Zerstörung von Wald- und Gehölzbiotopen,"<img src=""https://mymaps.usercontent.google.co...",POINT Z (6.8514 51.11003 0)
1,Zerstörung von von Geschützten Landschaftsbest...,"<img src=""https://mymaps.usercontent.google.co...",POINT Z (6.82848 51.10968 0)
2,Zerstörung von von Geschützten Landschaftsbest...,"<img src=""https://mymaps.usercontent.google.co...",POINT Z (6.82717 51.11007 0)
3,Zerstörung,"<img src=""https://mymaps.usercontent.google.co...",POINT Z (6.83104 51.10846 0)
4,Zerstörung von Wald- und Gehölzbiotopen (K3),"<img src=""https://mymaps.usercontent.google.co...",POINT Z (6.80947 51.11108 0)
5,Zerstörung von Wald- und Gehölzbiotopen (K 4),"<img src=""https://mymaps.usercontent.google.co...",POINT Z (6.76917 51.10389 0)
6,Untertunnelung,"<img src=""https://mymaps.usercontent.google.co...",POINT Z (6.73984 51.09539 0)
7,Untertunnelung,"<img src=""https://mymaps.usercontent.google.co...",POINT Z (6.72461 51.095 0)
8,Untertunnelung,"<img src=""https://mymaps.usercontent.google.co...",POINT Z (6.72149 51.09492 0)
9,Untertunnelung,"<img src=""https://mymaps.usercontent.google.co...",POINT Z (6.71884 51.09489 0)


In [780]:
folium.GeoJson(
    points, 
    name = "Destructions", # again, determine the name for the layer toggle
    pane = "pointsPane",
    marker=folium.Marker(
        icon=folium.Icon(color='lightred', icon='person-digging', prefix='fa')
    ),
    tooltip = folium.GeoJsonTooltip(fields=['Name'], labels = False)
).add_to(m)

In [781]:
folium.GeoJson(
    polys, 
    name = "Planned buildings", # again, determine the name for the layer toggle
    pane = "polygonsPane",
    style_function = lambda feature: {
        'color': 'black',
        'weight': 1,
        'fillColor': 'black',
        'fillOpacity': 0.5         
    },
    tooltip = folium.GeoJsonTooltip(fields=['Name'], labels = False)
).add_to(m)

In [782]:
# Add layer control to toggle layers
folium.LayerControl().add_to(m)
#m
m.show_in_browser()

Your map should have been opened in your browser automatically.
Press ctrl+c to return.


Error: Failed to open Wayland display, fallback to X11. WAYLAND_DISPLAY='wayland-0' DISPLAY=':0'


In [67]:
# save the map as hmtl
m.save('pipe_map.html')

---